In [1]:
import os
import cv2
import numpy as np
import numpy as np
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Create dictionary of object's Files

In [106]:
# Define the directory containing PCD files
data_dir = 'C:/Users/Neil Bonnard/Downloads/RGBD'

# Initialize an empty dictionary to store the dataset
dataset = {}

# Traverse through the directory structure
for instance_file in os.listdir(data_dir):
    if instance_file.endswith('a.pcd'):
        # Load PCD file using Open3D
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, instance_file))
        points = np.asarray(pcd.points)  # Extract x, y, z coordinates
        dataset[instance_file] = points

## Vizualize

In [107]:
# Visualize the first PCD file in the dictionary
if dataset:
    first_pcd_filename = list(dataset.keys())[0]
    first_pcd_points = dataset[first_pcd_filename]
    
    # Create an Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(first_pcd_points)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name=f"Visualization of {first_pcd_filename}")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L’opération de transformation demandée n’est pas prise en charge. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Descripteur non valide 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


## Compute shape:

In [108]:
from pyransac3d import Plane, Sphere, Cylinder, Cuboid
import numpy as np
import open3d as o3d

# Function to load and extract points
def load_and_extract_points(file_path):
    point_cloud = o3d.io.read_point_cloud(file_path)
    points = np.asarray(point_cloud.points)
    return points

# Function to fit shapes and print the one with the most inliers
def fit_and_print_shapes(points):
    shapes = {
        "plane": {"model": Plane(), "inliers": 0},
        "sphere": {"model": Sphere(), "inliers": 0},
        "cylinder": {"model": Cylinder(), "inliers": 0},
        "cuboid": {"model": Cuboid(), "inliers": 0}
    }
    
    # Fit a plane
    best_eq_plane, best_inliers_plane = shapes["plane"]["model"].fit(points, 0.004)
    shapes["plane"]["inliers"] = len(best_inliers_plane)
    
    # Fit a sphere
    center, radius, best_inliers_sphere = shapes["sphere"]["model"].fit(points, 0.004)
    shapes["sphere"]["inliers"] = len(best_inliers_sphere)
    
    # Fit a cylinder
    center, axis, radius, best_inliers_cylinder = shapes["cylinder"]["model"].fit(points, 0.004)
    shapes["cylinder"]["inliers"] = len(best_inliers_cylinder)
    
    # Fit a cuboid
    best_eq_cuboid, best_inliers_cuboid = shapes["cuboid"]["model"].fit(points, 0.004)
    shapes["cuboid"]["inliers"] = len(best_inliers_cuboid)
    
    # Determine shape with the most inliers
    best_shape = max(shapes, key=lambda shape: shapes[shape]["inliers"])
    num_inliers = shapes[best_shape]["inliers"]
    
    print(f"Shape detected: {best_shape} with {num_inliers} inliers")
    
    return shapes

# Initialize totals for each shape
total_inliers = {
    "plane": 0,
    "sphere": 0,
    "cylinder": 0,
    "cuboid": 0
}

# Process and print shapes for the first 100 PCD files in the dataset
file_limit = 100
processed_files = 0

for filename, points in dataset.items():
    if processed_files >= file_limit:
        break
    print(f"Fitting shapes for {filename}")
    shape_inliers = fit_and_print_shapes(points)
    
    # Accumulate the total inliers for each shape
    for shape, data in shape_inliers.items():
        total_inliers[shape] += data['inliers']
    
    processed_files += 1

# Print total inliers for each shape
print("\nTotal inliers across all processed files:")
for shape, inliers in total_inliers.items():
    print(f"{shape.capitalize()}: {inliers} inliers")


Fitting shapes for LocoBanana.pcd
Shape detected: sphere with 224 inliers

Total inliers across all processed files:
Plane: 154 inliers
Sphere: 224 inliers
Cylinder: 165 inliers
Cuboid: 195 inliers
